# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with
this is your SQL, and after that an SQL that can do what the user request.
Your Database is composed by a SQL database with some tables.
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order
just answer something nice and simple, maximum 10 words, asking him for something that
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
    "tableName": "Professors",
    "fields": [
        {
            "name": "professor_id",
            "type": "int"
        },
        {
            "name": "name",
            "type": "varchar"
        },
        {
            "name": "department",
            "type": "varchar"
        }
        {
            "name": "university",
            "type": "varchar"
        }
        {
            "name": "research_interests",
            "type": "text"
        }
    ]
}
"""})

old_context.append({'role':'system', 'content':"""                    
second table:
{
    "tableName": "Reviews",
    "fields": [
        {
            "name": "review_id",
            "type": "int"
        },
        {
            "name": "professor_id",
            "type": "int"
        },
        {
            "name": "student_name",
            "type": "varchar"
        },
        {
            "name": "review_date",
            "type": "date"
        },
        {
            "name": "rating",
            "type": "ing"
        },
        {
            "name": "comments",
            "type": "text"
        }
    ]
}  
"""})

old_context.append( {'role':'system', 'content':"""
third table:
{
    "tableName": "Institutes",
    "fields": [
        {
            "name": "institute_id",
            "type": "int"
        },
        {
            "name": "name",
            "type": "varchar"
        },
        {
            "name": "location",
            "type": "varchar"
        },
        {
            "name": "type",
            "type": "varchar"
        }
    ]
}                                 
"""})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
CREATE TABLE Professors (
    professor_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    department VARCHAR(100) NOT NULL,
    university VARCHAR(100) NOT NULL,
    research_interests TEXT
);
             
CREATE TABLE Reviews (
    review_id INT PRIMARY KEY AUTO_INCREMENT,
    professor_id INT,
    student_name VARCHAR(100) NOT NULL,
    review_date DATE NOT NULL,
    rating INT CHECK (rating BETWEEN 1 AND 5),
    comments TEXT,
    FOREIGN KEY (professor_id) REFERENCES Professors(professor_id)
);
             
CREATE TABLE Institutes (
    institute_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    location VARCHAR(100) NOT NULL,
    type VARCHAR(50) NOT NULL  -- E.g., 'University', 'Research Center', etc.
);            
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
INSERT INTO Professors (name, department, university, research_interests)
VALUES 
('Dr. Alice Johnson', 'Computer Science', 'Ohio University', 'Artificial Intelligence, Machine Learning'),
('Dr. John Smith', 'Physics', 'Harvard University', 'Quantum Mechanics, Astrophysics'),
('Dr. Emily Wong', 'Chemistry', 'MIT', 'Organic Chemistry, Nanotechnology');
                 
INSERT INTO Reviews (professor_id, student_name, review_date, rating, comments)
VALUES 
(1, 'Jane Doe', '2023-05-10', 4, 'Great teaching style but difficult exams.'),
(2, 'Mike Brown', '2023-06-20', 5, 'Very insightful lectures and helpful during office hours.'),
(3, 'Sarah White', '2023-07-15', 3, 'Knowledgeable but not very approachable.');
                 
INSERT INTO Institutes (name, location, type)
VALUES 
('Ohio University', 'Athens, OH', 'University'),
('Harvard University', 'Cambridge, MA', 'University'),
('MIT Research Center', 'Cambridge, MA', 'Research Center');"""
})

In [5]:
# Function to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    context = context.copy()
    context.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [6]:
question = "Which professor from the physics department got the best review?"

In [7]:
# new
context_user = context.copy()
print(return_CCRMSQL(question, context_user))

To find out which professor from the Physics department received the best review, we need to consider the reviews' ratings for professors in the Physics department.

We can achieve this by joining the `Professors` and `Reviews` tables on the `professor_id` column and filtering for professors in the Physics department. Then, we can order the results by the rating in descending order and limit the result to the top row to get the professor with the best review.

Here is the SQL query to find the professor from the Physics department who received the best review:

```sql
SELECT p.name AS professor_name, r.rating AS review_rating
FROM Professors p
JOIN Reviews r ON p.professor_id = r.professor_id
WHERE p.department = 'Physics'
ORDER BY r.rating DESC
LIMIT 1;
```

When you run this query, it will return the name of the professor from the Physics department who received the best review along with the rating they received.


In [8]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL(question, old_context_user))

This is your SQL:
```sql
SELECT p.name, AVG(r.rating) AS avg_rating
FROM Professors p
JOIN Reviews r ON p.professor_id = r.professor_id
WHERE p.department = 'Physics'
GROUP BY p.name
ORDER BY avg_rating DESC
LIMIT 1;
```

This SQL query selects the name of the professor from the Physics department who received the highest average rating in reviews. It joins the Professors and Reviews tables on the professor_id, filters by the Physics department, calculates the average rating for each professor, orders the results in descending order of average rating, and limits the output to the top result.


In [12]:
question = "In which institute can we find the professors with the worst reviews?"

In [13]:
#new
print(return_CCRMSQL(question, context_user))

To find the institute where professors with the worst reviews are located, we need to consider the professors' reviews and their associated institutes. We can achieve this by first identifying the professor(s) with the lowest average rating and then determining the institute(s) they belong to.

Here is the SQL query to find the institute(s) where professors with the worst reviews are located:

```sql
WITH ProfessorAvgRating AS (
    SELECT professor_id, AVG(rating) AS avg_rating
    FROM Reviews
    GROUP BY professor_id
)
SELECT i.name AS institute_name
FROM Professors p
JOIN Reviews r ON p.professor_id = r.professor_id
JOIN Institutes i ON p.university = i.name
WHERE p.professor_id IN (
    SELECT professor_id
    FROM ProfessorAvgRating
    WHERE avg_rating = (
        SELECT MIN(avg_rating)
        FROM ProfessorAvgRating
    )
);
```

This query calculates the average rating for each professor, then selects the institute(s) where the professor(s) with the lowest average rating are

In [14]:
#old
print(return_CCRMSQL(question, old_context_user))

this is your SQL:
```sql
SELECT DISTINCT I.name AS institute_name
FROM Professors P
JOIN Reviews R ON P.professor_id = R.professor_id
JOIN Institutes I ON P.university = I.name
GROUP BY I.name
ORDER BY AVG(R.rating) ASC
LIMIT 1;
```

This SQL query retrieves the name of the institute where you can find the professors with the worst reviews. It joins the Professors, Reviews, and Institutes tables, calculates the average rating for each institute, orders the results in ascending order of average rating, and then selects the institute with the lowest average rating.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [17]:
def compare_sql_queries(question: str) -> None: 
    """Print both completions from the old context and the new one
    generated from the same question.

    Args:
        question (str): question
    """

    # old context
    old_context_user = old_context.copy()
    print(return_CCRMSQL(question, old_context_user))
    
    print(100*'-')

    # new context
    context_user = context.copy()
    print(return_CCRMSQL(question, context_user))

### Variation 1

In [19]:
question = "I need the name and department of the student that has reviewed the most professors"
compare_sql_queries(question)

this is your SQL:
```sql
SELECT student_name, COUNT(professor_id) AS num_reviews
FROM Reviews
GROUP BY student_name
ORDER BY num_reviews DESC
LIMIT 1;
```

This SQL query selects the student_name and counts the number of reviews they have made for professors. It then groups the results by student_name, orders them in descending order based on the number of reviews, and limits the output to only the student with the highest number of reviews.
----------------------------------------------------------------------------------------------------
```sql
WITH StudentReviews AS (
    SELECT student_name, COUNT(DISTINCT professor_id) AS num_professors_reviewed
    FROM Reviews
    GROUP BY student_name
)
SELECT student_name
FROM StudentReviews
WHERE num_professors_reviewed = (SELECT MAX(num_professors_reviewed) FROM StudentReviews);
```


**Old context:**  
This query is simple but likely inaccurate without counting distinct professors, which is the main drawback.

**New context:**  
This query is accurate and well-structured, though slightly more complex. The use of COUNT(DISTINCT) and a CTE addresses the problem requirements thoroughly.

### Variation 2

In [20]:
question = "What institute holds the best reviewed teachers?"
compare_sql_queries(question)

this is your SQL:
```sql
SELECT Institutes.name
FROM Institutes
JOIN Professors ON Institutes.institute_id = Professors.institute_id
JOIN Reviews ON Professors.professor_id = Reviews.professor_id
GROUP BY Institutes.name
ORDER BY AVG(Reviews.rating) DESC
LIMIT 1;
```

This SQL query retrieves the name of the institute that holds the best-reviewed teachers. It joins the Institutes, Professors, and Reviews tables, calculates the average rating for each institute, and then selects the institute with the highest average rating.
----------------------------------------------------------------------------------------------------
To determine which institute holds the best reviewed teachers, we need to consider the average ratings of the professors at each institute. We can achieve this by joining the `Professors` and `Reviews` tables on the `professor_id` column, then grouping by the `university` and calculating the average rating for each university. Finally, we can order the results by the

**Old context:**  
This query is both accurate and clear, fulfilling the requirement to identify the institute with the best-reviewed teachers.

**New context:**  
This query is clear but misses the exact answer, as it does not limit the results to the single best-reviewed institute. Adding LIMIT 1 would improve its accuracy.

### Variation 3

In [21]:
question = "Some students have not reviewed any professor yet. I want their names to ban them from the University."
compare_sql_queries(question)

this is your SQL:
```sql
SELECT student_name
FROM Students
LEFT JOIN Reviews ON Students.student_name = Reviews.student_name
WHERE Reviews.student_name IS NULL;
```

This SQL query selects the names of students who have not reviewed any professor yet by performing a LEFT JOIN between the Students table and the Reviews table on the student_name column. It then filters the results to only include rows where there is no matching review for the student, indicating they have not reviewed any professor.
----------------------------------------------------------------------------------------------------
To find the names of students who have not reviewed any professor yet, you can use the following SQL query:

```sql
SELECT DISTINCT student_name
FROM Reviews
WHERE student_name NOT IN (SELECT DISTINCT student_name FROM Reviews);
```

This query will return the names of students who have not reviewed any professor yet. You can then take appropriate action based on this information.


**Old context:**  
This query is accurate, efficient, and highly readable, effectively solving the problem.

**New context:**  
This query is incorrect in logic, inefficient, and less readable due to its flawed approach.

## CONCLUSION
Both context seem to yield similar results, one being sometimes better than the other. The examples provided in the second context probably helps the model come up with a better solution sometimes (see variation 1) but not necessarely all the time (see variations 2 and 3). As usual, adding an extra layer of complexity may confuse the model.